In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as soup
import pandas as pd

# Set up Splinter
browser = Browser('chrome')

# Create an empty list to store the TV show data
TVshows = []

# Loop over all the URLs
for i in range(10):
    # Construct the URL for the current page
    url = f'https://www.imdb.com/search/title/?title_type=tv_series&num_votes=1000,&languages=en&sort=num_votes,desc&count=100&start={i*100+1}&ref_=adv_nxt'

    # Visit the URL
    browser.visit(url)

    # Parse the website
    html = browser.html
    html_soup = soup(html, 'html.parser')

    # Find the list of TV shows
    TV_list = html_soup.find('div', {'class': 'lister-list'})

    # Find all the rows in the list
    TV_rows = TV_list.find_all('div', {'class': 'lister-item'})

    # Loop over the rows and extract the TV show details
    for row in TV_rows:
        # Get the TV show title
        title = row.find('h3', {'class': 'lister-item-header'}).find('a').text

        # Get the TV show year
        year = row.find('span', {'class': 'lister-item-year'}).text.strip('()')

        # Get the TV show rating
        rating = row.find('div', {'class': 'ratings-bar'}).find('strong').text

        # Get the votes and stars
        details = row.find('div', {'class': 'lister-item-content'})
        stars = [star.text for star in details.find_all('p')[2].find_all('a')[1:]]
        votes = details.find('p', {'class': 'sort-num_votes-visible'}).find_all('span')[1].text.replace(',', '')

        # Get the TV show genres
        genres = [genre.strip() for genre in details.find('span', {'class': 'genre'}).text.split(',')]
        
        # Add the TV show data to the list
        TVshows.append({'Title': title, 'Year': year, 'Rating': rating, 'Votes': votes, 'Stars': stars, 'Genres': genres})


# Create a DataFrame from the TV show data
df_TV = pd.DataFrame(TVshows)

print(df_TV)

# Export the DataFrame to a CSV file
df_TV.to_csv('imdb_top_1000_TV_Final.csv', index=False)


                                  Title       Year Rating    Votes  \
0                       Game of Thrones  2011–2019    9.2  2140341   
1                          Breaking Bad  2008–2013    9.5  1944497   
2                       Stranger Things  2016–2024    8.7  1224826   
3                      The Walking Dead  2010–2022    8.1  1016107   
4                               Friends  1994–2004    8.9  1015472   
..                                  ...        ...    ...      ...   
995  Two Guys, a Girl and a Pizza Place  1998–2001    7.4    17350   
996                       Metalocalypse  2006–2013    8.3    17294   
997                           Uncoupled       2022    7.0    17273   
998            The Ellen DeGeneres Show  2003–2023    6.7    17246   
999                           Dickinson  2019–2021    7.6    17242   

                                                 Stars  \
0         [Peter Dinklage, Kit Harington, Lena Headey]   
1                [Aaron Paul, Anna Gunn, Be